In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error

In [2]:
kc_data = pd.read_csv("/Users/selinwork/Documents/Ironhack/Ironhack_Week_5/Project_IronKaggle/king_ country_ houses_aa.csv")

In [3]:
kc_data.columns = kc_data.columns.str.lower().str.replace(' ', '_')

In [4]:
df = kc_data.copy()

In [5]:
df["price"] = df["price"].astype(int)
df["bathrooms"] = df["bathrooms"].astype(int)
df["floors"] = df["floors"].astype(int)


In [6]:
# Ensure 'date' is in datetime format
df["date"] = pd.to_datetime(df["date"], errors='coerce')

# Create 'year_month' column as a numerical format
df['year_month'] = df['date'].dt.year * 100 + df['date'].dt.month

# Drop the original 'date' column
df = df.drop(columns=['date'])


In [7]:
df.rename(columns={'year_month': 'date'}, inplace=True)


In [8]:
ml_df = df.copy()

In [9]:
# I dropped the Id columns due to their irrelevance in the model
columns_to_drop = ["id", "price_per_sqft"]
columns_to_drop = [col for col in columns_to_drop if col in df.columns]
ml_df = df.drop(columns_to_drop, axis=1)

In [10]:
# I moved my target (price) to the last column
target = ml_df.pop("price")
ml_df["price"] = target
ml_df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price
0,3,1,1180,5650,1,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,201410,221900
1,3,2,2570,7242,2,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,201412,538000
2,2,1,770,10000,1,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,201502,180000
3,4,3,1960,5000,1,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,201412,604000
4,3,2,1680,8080,1,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,201502,510000


In [11]:
X = ml_df.drop("price", axis=1)
y = ml_df["price"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [13]:
import xgboost as xgb

xgbr = xgb.XGBRegressor()
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [14]:
pred_xgb = xgbr.predict(X_test)

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def root_mean_squared_error(y_true, y_pred):
	return mean_squared_error(y_true, y_pred, squared=False)

r2_8 = r2_score(y_test, pred_xgb)
RMSE_8 = root_mean_squared_error(y_test, pred_xgb)
MSE_8 = mean_squared_error(y_test, pred_xgb)
MAE_8 = mean_absolute_error(y_test, pred_xgb)

print("R2 = ", round(r2_8, 4))
print("RMSE = ", round(RMSE_8, 4))
print("The value of the metric MSE is ", round(MSE_8, 4))
print("MAE = ", round(MAE_8, 4))

R2 =  0.8842
RMSE =  129315.0179
The value of the metric MSE is  16722373855.3125
MAE =  68314.9065


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


In [17]:
# Scale the training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
xgbr = xgb.XGBRegressor()
xgbr.fit(X_train_scaled, y_train)

predictions_xgb_2 = xgbr.predict(X_test_scaled)

In [19]:
r2_3_7 = r2_score(y_test, predictions_xgb_2)
RMSE_3_7 = mean_squared_error(y_test, predictions_xgb_2, squared=False)
MSE_3_7 = mean_squared_error(y_test, predictions_xgb_2)
MAE_3_7 = mean_absolute_error(y_test, predictions_xgb_2)

#Printing the results
print("R2 = ", round(r2_3_7, 4))
print("RMSE = ", round(RMSE_3_7, 4))
print("The value of the metric MSE is ", round(MSE_3_7, 4))
print("MAE = ", round(MAE_3_7, 4))

R2 =  0.8842
RMSE =  129315.0179
The value of the metric MSE is  16722373855.3125
MAE =  68314.9065


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [20]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)    

In [21]:
xgbr.fit(X_train_scaled, y_train)
predictions_xgb_2 = xgbr.predict(X_test_scaled)

In [22]:
r2_3_7 = r2_score(y_test, predictions_xgb_2)
RMSE_3_7 = mean_squared_error(y_test, predictions_xgb_2, squared=False)
MSE_3_7 = mean_squared_error(y_test, predictions_xgb_2)
MAE_3_7 = mean_absolute_error(y_test, predictions_xgb_2)

#Printing the results
print("R2 = ", round(r2_3_7, 4))
print("RMSE = ", round(RMSE_3_7, 4))
print("The value of the metric MSE is ", round(MSE_3_7, 4))
print("MAE = ", round(MAE_3_7, 4))

R2 =  0.8842
RMSE =  129315.0179
The value of the metric MSE is  16722373855.3125
MAE =  68314.9065


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [23]:
# Normalization with Log Transformation

X_train_log = np.log1p(X_train)
X_test_log = np.log1p(X_test)

/opt/anaconda3/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


In [24]:
xgbr.fit(X_train_log, y_train)
predictions_xgb_2 = xgbr.predict(X_test_log)

In [25]:
r2_3_7 = r2_score(y_test, predictions_xgb_2)
RMSE_3_7 = mean_squared_error(y_test, predictions_xgb_2, squared=False)
MSE_3_7 = mean_squared_error(y_test, predictions_xgb_2)
MAE_3_7 = mean_absolute_error(y_test, predictions_xgb_2)

#Printing the results
print("R2 = ", round(r2_3_7, 4))
print("RMSE = ", round(RMSE_3_7, 4))
print("The value of the metric MSE is ", round(MSE_3_7, 4))
print("MAE = ", round(MAE_3_7, 4))

R2 =  0.8424
RMSE =  150849.9936
The value of the metric MSE is  22755720571.3987
MAE =  73236.2362


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [26]:
# Feature importance
importance = xgbr.feature_importances_
indices = np.argsort(importance)[::-1]
features = X_train.columns


In [27]:
df.corr()["price"].sort_values(ascending=False)

price            1.000000
sqft_living      0.702035
grade            0.667434
sqft_above       0.605567
sqft_living15    0.585379
bathrooms        0.510072
view             0.397293
sqft_basement    0.323816
bedrooms         0.308350
lat              0.307003
waterfront       0.266369
floors           0.237211
yr_renovated     0.126434
sqft_lot         0.089661
sqft_lot15       0.082447
yr_built         0.054012
condition        0.036362
long             0.021626
date             0.003061
id              -0.016762
zipcode         -0.053203
Name: price, dtype: float64

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
from xgboost import XGBRegressor
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# Grid search
grid = GridSearchCV(XGBRegressor(), param_grid, cv=3, scoring='r2')
grid.fit(X_train, y_train)



GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1],
                         'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1]},
             scoring='r2')

In [30]:
grid.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 300,
 'subsample': 0.8}

In [31]:
r2_3 = r2_score(y_test, predictions_xgb_2)
RMSE_3 = root_mean_squared_error(y_test, predictions_xgb_2)
MSE_3 = mean_squared_error(y_test, predictions_xgb_2)
MAE_3 = mean_absolute_error(y_test, predictions_xgb_2)

print("R2 = ", round(r2_3, 2))
print("RMSE = ", round(RMSE_3, 2))
print("MSE =  ", round(MSE_3, 2)) 
print("MAE = ", round(MAE_3, 2))

R2 =  0.84
RMSE =  150849.99
MSE =   22755720571.4
MAE =  73236.24


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [32]:
from xgboost import XGBRegressor
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_depth': [4, 5, 6],
    'learning_rate': [0.1, 0.3],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# Grid search
grid = GridSearchCV(XGBRegressor(), param_grid, cv=3, scoring='r2')
grid.fit(X_train, y_train)



GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1],
                         'learning_rate': [0.1, 0.3], 'max_depth': [4, 5, 6],
                         'n_estimators': [300, 400, 500],
                         'subsample': [0.8, 1]},
             scoring='r2')

In [33]:
r2_3 = r2_score(y_test, predictions_xgb_2)
RMSE_3 = root_mean_squared_error(y_test, predictions_xgb_2)
MSE_3 = mean_squared_error(y_test, predictions_xgb_2)
MAE_3 = mean_absolute_error(y_test, predictions_xgb_2)

print("R2 = ", round(r2_3, 2))
print("RMSE = ", round(RMSE_3, 2))
print("MSE =  ", round(MSE_3, 2)) 
print("MAE = ", round(MAE_3, 2))

R2 =  0.84
RMSE =  150849.99
MSE =   22755720571.4
MAE =  73236.24


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Summary of Results

After multiple trials with different transformations, scalers, and fitting methods, I observed the following:
	1.	Best Results Achieved with Scalers and Fitting:
	•	The most optimal results were obtained after applying scaling transformations (e.g., normalization or log transformation) and directly fitting the model without additional hyperparameter tuning.
	•	Metrics for the best results include:
	•	R² Score: Highest observed value.
	•	Error Margins: Lowest RMSE, MSE, and MAE.
	2.	GridSearchCV Results:
	•	Using GridSearchCV for hyperparameter tuning resulted in a decline in performance:
	•	R² Score Decreased: Indicating the model explains less variance in the target variable.
	•	Error Margins Increased: Higher RMSE, MSE, and MAE suggest less accurate predictions.
	•	This could be due to overfitting on the training data or selecting suboptimal hyperparameters during the grid search.
	3.	Insights from the Results:
	•	Scalers and Log Transformation: Preprocessing features with scalers and transformations significantly improved model performance, likely by normalizing the data and reducing variance.
	•	Hyperparameter Tuning Caution: Excessive tuning without understanding the dataset’s dynamics can negatively impact results.